In [1]:
import struct
from enum import Enum
from bitarray import bitarray

In [5]:
# Initial key permutation PC-1 (permuted choice one)
PC1 = [57, 49, 41, 33, 25, 17,  9,  1,
       58, 50, 42, 34, 26, 18, 10,  2,
       59, 51, 43, 35, 27, 19, 11,  3,
       60, 52, 44, 36, 63, 55, 47, 39,
       31, 23, 15,  7, 62, 54, 46, 38,
       30, 22, 14,  6, 61, 53, 45, 37,
       29, 21, 13,  5, 28, 20, 12,  4]

# Round key permutation PC-2 (permuted choice two)
PC2 = [14, 17, 11, 24,  1,  5,  3, 28,
       15,  6, 21, 10, 23, 19, 12,  4,
       26,  8, 16,  7, 27, 20, 13,  2,
       41, 52, 31, 37, 47, 55, 30, 40,
       51, 45, 33, 48, 44, 49, 39, 56,
       34, 53, 46, 42, 50, 36, 29, 32]

In [7]:
class DES_words(Enum):
    L = 0
    R = 0

class DES_tables(Enum):
    # Initial permutation
    IP =     [58, 50, 42, 34, 26, 18, 10, 2,
              60, 52, 44, 36, 28, 20, 12, 4,
              62, 54, 46, 38, 30, 22, 14, 6,
              64, 56, 48, 40, 32, 24, 16, 8,
              57, 49, 41, 33, 25, 17, 9,  1,
              59, 51, 43, 35, 27, 19, 11, 3,
              61, 53, 45, 37, 29, 21, 13, 5,
              63, 55, 47, 39, 31, 23, 15, 7]
    
    # Final permutation
    IP_inv = [40, 8, 48, 16, 56, 24, 64, 32,
              39, 7, 47, 15, 55, 23, 63, 31,
              38, 6, 46, 14, 54, 22, 62, 30,
              37, 5, 45, 13, 53, 21, 61, 29,
              36, 4, 44, 12, 52, 20, 60, 28,
              35, 3, 43, 11, 51, 19, 59, 27,
              34, 2, 42, 10, 50, 18, 58, 26,
              33, 1, 41, 9,  49, 17, 57, 25]
    
    # Expansion box for the data
    E =      [32,  1,  2,  3,  4,  5,
               4,  5,  6,  7,  8,  9,
               8,  9, 10, 11, 12, 13,
              12, 13, 14, 15, 16, 17,
              16, 17, 18, 19, 20, 21,
              20, 21, 22, 23, 24, 25,
              24, 25, 26, 27, 28, 29,
              28, 29, 30, 31, 32,  1]
    
    # Permutation P within the f-function
    P = [16,  7, 20, 21, 29, 12, 28, 17,
          1, 15, 23, 26,  5, 18, 31, 10,
          2,  8, 24, 14, 32, 27,  3,  9,
         19, 13, 30,  6, 22, 11,  4, 25]
    
    
    # S-boxes for the data
    S = [
        [[14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
         [ 0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8],
         [ 4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0],
         [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]],

        [[15,  1,  8, 14,  6, 11,  3,  4,  9,  7,  2, 13, 12,  0,  5, 10],
         [ 3, 13,  4,  7, 15,  2,  8, 14, 12,  0,  1, 10,  6,  9, 11,  5],
         [ 0, 14,  7, 11, 10,  4, 13,  1,  5,  8, 12,  6,  9,  3,  2, 15],
         [13,  8, 10,  1,  3, 15,  4,  2, 11,  6,  7, 12,  0,  5, 14,  9]],

        [[10,  0,  9, 14,  6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8],
         [13,  7,  0,  9,  3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1],
         [13,  6,  4,  9,  8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7],
         [ 1, 10, 13,  0,  6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12]],

        [[ 7, 13, 14,  3,  0,  6,  9, 10,  1,  2,  8,  5, 11, 12,  4, 15],
         [13,  8, 11,  5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,  9],
         [10,  6,  9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4],
         [ 3, 15,  0,  6, 10,  1, 13,  8,  9,  4,  5, 11, 12,  7,  2, 14]],

        [[ 2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9],
         [14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3,  9,  8,  6],
         [ 4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14],
         [11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3]],

        [[12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11],
         [10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8],
         [ 9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6],
         [ 4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13]],

        [[ 4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1],
         [13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6],
         [ 1,  4, 11, 13, 12,  3,  7, 14, 10, 15,  6,  8,  0,  5,  9,  2],
         [ 6, 11, 13,  8,  1,  4, 10,  7,  9,  5,  0, 15, 14,  2,  3, 12]],

        [[13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7],
         [ 1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2],
         [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8],
         [ 2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11]]
    ]



class DES(object):
    _string = None
    _words = {
        DES_words.L: 0,
        DES_words.R: 0,
    }
    
    @classmethod
    def encrypt(cls, string):
        cls._string = string
        padded_bit_array = cls._padding(cls._string2bits())
        
        N = len(padded_bit_array) // 64
        for chunk_index in range(N):
            start = chunk_index * 64
            chunked_bit_array = padded_bit_array[start:start+64]
            
            # Convert 'bitarray' objects to integers.
            X = int.from_bytes(chunked_bit_array.tobytes(), byteorder="little")
            
            # Apply initial permutation
            Y = cls._init_perm(X)
            
            # Fiestel network for 16 rounds
            cls._rounds(Y)
        return Y
#         cls._rounds()
#         return cls._final_perm()
    
    @classmethod
    def _string2bits(cls):
        # Convert string to a bit array.
        bit_array = bitarray(endian="little")
        bit_array.frombytes(cls._string.encode("utf-8"))
        return bit_array
    
    @classmethod
    def _padding(cls, bit_array):
        # "OneAndZeroes" padding
        bit_array.append(1)
        while bit_array.length() % 64 != 0:
            bit_array.append(0)
        return bit_array
    
    @classmethod
    def _init_perm(cls, x):
        # Initial permutation for the data
        IP = DES_tables.IP.value
        p = [i - 1 for i in IP]
        return perm(x, p)
    
    # DES rounds
    @classmethod
    def _rounds(cls, Y):
        # Define 32-bit splitting function
        def word_split(Y):
            R = Y & 0xffffffff
            L = (Y >> 32) & 0xffffffff
            return [L, R]
        
        # Demultiplex 48 bytes into eight 6-bit blocks
        def demux(x):
            y = []
            for i in range(8):
                y.append((x >> i) & 0x3f)
            return y
        
        # Multiplex eight 4-bit blocks into a 32-bit block
        def mux(x):
            y = 0
            for i, u in enumerate(x):
                y = y ^ (u << (i * 4))
            return y
        
        def sbox(box_num, x):
            S = DES_tables.S.value
            # Row and column select
            r, c = 0, 0
            r = r ^ (x & 1)
            c = c ^ ((x >> 1) & 15)
            r = r ^ ((x >> 4) & 2)
            return S[box_num][r][c]
        
        # Define f-function
        def F(R, k):
            # Expansion block
            p = [i - 1 for i in DES_tables.E.value]
            R_exp = exp(R, p)
            
            # XOR with the subkey
            Q = R_exp ^ k
            
            # Demultiplex into eight 6-bit blocks
            U = demux(Q)
            
            # Pass through substitution boxes S1-S8
            # Note: MSB block goes in first
            W = []
            V = [t for t in reversed(U)]
            for i, v in enumerate(V):
                W.append(sbox(i, v))
                
            # Join the eight 4-bit boxes
            Z = mux(W)
            
            # F-function permutation
            P = DES_tables.P.value
            p = [i - 1 for i in P]
            F = perm(Z, p)
            
            return None
        
        # Define subkey function for n rounds
#         def subkey(K, n):
#             return 1
        
        
        # Split the input into 2 halves
        [L0, R0] = word_split(Y)
        cls._words[DES_words.L] = L0
        cls._words[DES_words.R] = R0
        
        return None
    
    def _final_perm(cls):
        return None

def perm(x, p):
    y = 0
    for q in range(len(p)):
        if (x & (1 << q)) != 0:
            y = y ^ (1 << p[q])
    return y

def exp(x, p):
    B = bitarray(endian="big")
    bits = bin(x)[2:]
    for i in p:
        B.append(int(bits[i]))
    return int.from_bytes(B.tobytes(), byteorder="big")

In [4]:
y = DES.encrypt("b")
y

577023702256976384

In [10]:
P = DES_tables.P.value
p = [i - 1 for i in P]
F = perm(1, p)
bin(F)[2:]

'1000000000000000'

In [11]:
P

[16,
 7,
 20,
 21,
 29,
 12,
 28,
 17,
 1,
 15,
 23,
 26,
 5,
 18,
 31,
 10,
 2,
 8,
 24,
 14,
 32,
 27,
 3,
 9,
 19,
 13,
 30,
 6,
 22,
 11,
 4,
 25]